<a href="https://colab.research.google.com/github/lesya7maltseva/Pandas-Statistics-/blob/main/Pandas_rolling%2C_cumcountum%2C_shift_%D0%A1%D0%BA%D0%BE%D0%BB%D1%8C%D0%B7%D1%8F%D1%89%D0%B5%D0%B5_%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузите файл в датафреймы Pandas и назовите его coderun. Создайте в этом датафрейме столбец с датой и назовите его date.

Далее создайте новый датафрейм df, который получается в результате:
отсечения всех дат до 01.01.2022
группировки по дате
расчету количества задач, решаемых в каждый день (столбец назовите problems)
«превращение» столбца с датой из индекса в обычный столбец

А далее необходимо выполнить действия, типичные для работы, например, с временными рядами.

Создайте столбец rolling_avg, который получается усреднения столбца problems по 7 соседним измерениям. При этом усреднение происходит не по последним 7, а по:

предыдущим 3 элементам
текущему элементу
следующим 3 элементам
Возникшие в результате пропуске замените на 0. Значения округлите до 2 знака после запятой.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
coderun = pd.read_csv('itresume-coderun.csv')
coderun.head(5)

,id,created_at,problem_id,user_id,language_id
0,1,2021-04-07 06:06:20.000,13,10,3
1,2,2021-03-31 07:10:06.000,15,13,3
2,3,2021-04-04 14:55:26.000,1,6,3
3,4,2021-03-29 21:24:51.000,26,4,3
4,5,2021-03-30 11:29:12.000,21,18,3


In [ ]:
coderun['date'] = pd.to_datetime(coderun['created_at']).dt.date
df = coderun.loc[pd.to_datetime(coderun['date']).dt.strftime('%Y-%m-%d') >= '2022-01-01']


,id,created_at,problem_id,user_id,language_id,date
7333,7334,2022-01-01 00:03:34.826,54,342,3,2022-01-01
7334,7335,2022-01-01 00:04:13.721,54,342,3,2022-01-01
7335,7336,2022-01-01 00:16:28.729,48,342,3,2022-01-01
7336,7337,2022-01-01 00:16:44.156,48,342,3,2022-01-01
7337,7338,2022-01-01 00:17:25.300,48,342,3,2022-01-01
...,...,...,...,...,...,...
55717,55718,2022-05-17 09:00:51.743,101,171,2,2022-05-17
55718,55719,2022-05-17 09:01:04.210,101,171,2,2022-05-17
55719,55720,2022-05-17 09:02:06.203,101,171,2,2022-05-17
55720,55721,2022-05-17 09:03:45.265,102,171,2,2022-05-17


In [ ]:
df = df.groupby('date').agg({'problem_id':'count'}).reset_index()
df = df.rename(columns={'problem_id':'problems'})


In [ ]:
df['rolling_avg'] = (
    df['problems']
    .rolling(7, center=True)
    .mean()
    .apply(lambda x: np.round(x, 2))
    .fillna(0)
    )

In [ ]:
df

,created_at,problems,rolling_avg
0,2022-01-01,1,0.0
1,2022-01-02,1,0.0
2,2022-01-03,1,0.0
3,2022-01-04,1,1.0
4,2022-01-05,1,1.0
...,...,...,...
131,2022-05-13,1,1.0
132,2022-05-14,1,1.0
133,2022-05-15,1,0.0
134,2022-05-16,1,0.0


Загрузите файл в датафреймы Pandas и назовите его coderun.
Создайте в датафрейме coderun столбец rnk, который ведет себя как оконная функция RANK в SQL - ранжирует строки по заданным правилам. Правила такие:
ранжирование строк должно быть отдельным в рамках каждого пользователя
ранжирование происходит на основании номера задачи и даты-времени ее решения
если пользователь и задача совпадают, принимаем решение на основании даты-времени
все сортировки берем по возрастанию

После этого создайте датафрейм df:
оставьте только ранги, которые больше 3
по всем рангам посчитайте количество уникальных пользователей, у которых встречался данный ранг (назовите столбец amount)

столбец с рангом должен быть не в индексе
А в конце добавьте столбец diff, который покажет процентную динамику между текущим amount и предыдущим. Возникшие в результате пропуске замените на 0. Значения округлите до 2 знака после запятой.

In [4]:
coderun = pd.read_csv('itresume-coderun.csv')
coderun.head(5)

,id,created_at,problem_id,user_id,language_id
0,1,2021-04-07 06:06:20.000,13,10,3
1,2,2021-03-31 07:10:06.000,15,13,3
2,3,2021-04-04 14:55:26.000,1,6,3
3,4,2021-03-29 21:24:51.000,26,4,3
4,5,2021-03-30 11:29:12.000,21,18,3


In [26]:
#аналог sql rank для панд
coderun['rnk'] = (
    coderun
    .sort_values(['user_id','created_at','problem_id'])
    .groupby('user_id')
    .cumcount()+1
    )



In [33]:
df = (
    coderun[coderun['rnk'] > 3]
    .groupby('rnk')['user_id']
    .agg(amount = 'nunique')
    .reset_index()
)
df['diff'] = df['amount'].pct_change().fillna(0).round(2)
df

,rnk,amount,diff
0,4,660,0.00
1,5,625,-0.05
2,6,595,-0.05
3,7,569,-0.04
4,8,543,-0.05
...,...,...,...
1458,1462,1,0.00
1459,1463,1,0.00
1460,1464,1,0.00
1461,1465,1,0.00


Первичные действия:

Прочитайте содержимое файла без участия Pandas в переменную f.
Сформируйте список lst, который будет состоять из списков, где каждый список представляет собой строку файла. Длина каждого списка равняется количеству столбцов в файле.
Если в названиях столбцов присутсвуют лишние символы - уберите их.
Если вместе с данными загрузились полностью пустые строки - уберите их.
В конце создайте датафрейм Pandas и назовите его df. Названия столбцов и содержимое возьмите из созданного ранее списка lst.

In [ ]:
lst = []
with open('itresume-coderun.csv', 'r') as f:
  for line in f:
    lst.append(line.strip('\n').split(','))
columns = [el.strip('","\n') for el in lst[0]]
lst.pop(0)
df = pd.DataFrame(lst, columns=columns)
df

Загрузите файл в датафреймы Pandas и назовите его coderun. Создайте в этом датафрейме столбец с датой и назовите его date.

Далее создайте новый датафрейм df, который получается в результате:
группировки исходного датафрейма по дате
расчету количества уникальных задач, решаемых в каждый день (столбец назовите problems)
«превращение» столбца с датой из индекса в обычный столбец

А далее необходимо выполнить действия, типичные для работы, например, с временными рядами.
Допустим, вы провели исследование и вас смущает автокорреляция 3 порядка. Вам необходимо:

Создать столбец prev_problems и записать в него значения столбца problems, сдвинутые на 3 назад. Если есть пропуски - туда вставляем 0.

Создать столбец diff и построчно посчитать разницу между problems и prev_problems.

In [52]:
coderun = pd.read_csv('itresume-coderun.csv')
coderun['date'] = pd.to_datetime(coderun['created_at']).dt.date
df = coderun.groupby(['date'])['problem_id'].agg(problems = 'nunique').reset_index()
df['prev_problems'] = df['problems'].shift(3, fill_value=0)
df['diff'] = df['problems'] - df['prev_problems']

In [58]:
df

,date,problems,prev_problems,diff
0,2021-03-27,1,0,1
1,2021-03-28,1,0,1
2,2021-03-29,2,0,2
3,2021-03-30,1,1,0
4,2021-03-31,2,1,1
...,...,...,...,...
242,2022-05-13,41,24,17
243,2022-05-14,29,56,-27
244,2022-05-15,32,51,-19
245,2022-05-16,25,41,-16
